# Пример использования модульного чат-бота

В этом ноутбуке показано, как использовать модульный чат-бот с Retrieval-Augmented Generation (RAG).

In [1]:
import os
import sys
from dotenv import load_dotenv

# Добавляем директорию с модулями в путь поиска
sys.path.append('..')

# Загружаем переменные окружения из .env
load_dotenv()

True

## 1. Импорт необходимых модулей

In [2]:
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document

from chatbot_modules import RAGChatBot

## 2. Загрузка векторного хранилища и создание ретривера

In [3]:
# Загрузка модели эмбеддингов
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Загрузка векторного хранилища
vector_store = Chroma(
    collection_name="knowledge_markdown",
    embedding_function=embeddings,
    persist_directory="../chroma_langchain_db/knowledge"
)

# Создание ретривера по векторной близости
similarity_retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# Извлечение всех документов для BM25 ретривера
raw_collection = vector_store._collection.get()
docs = []

# Проверка наличия документов
if "documents" in raw_collection and raw_collection["documents"]:
    for content, meta in zip(raw_collection["documents"], raw_collection["metadatas"]):
        docs.append(Document(page_content=content, metadata=meta))
    
    # Создание BM25 ретривера только если есть документы
    if docs:
        keyword_retriever = BM25Retriever.from_documents(docs)
        keyword_retriever.k = 5
        
        # Создание ансамбля ретриверов
        ensemble_retriever = EnsembleRetriever(
            retrievers=[similarity_retriever, keyword_retriever],
            weights=[0.5, 0.5]
        )
    else:
        print("Предупреждение: Нет документов для BM25Retriever, используется только векторный поиск")
        ensemble_retriever = similarity_retriever
else:
    print("Предупреждение: Нет документов в векторном хранилище, используется только векторный поиск")
    ensemble_retriever = similarity_retriever

/var/folders/h2/r9wh0x750xq4v5z33ylwrzw40000gn/T/ipykernel_62093/1143437512.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/Users/sergey/Desktop/Voise_RAG/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3. Создание языковой модели

In [4]:
# Проверка наличия API ключа
api_key = os.getenv("OPENROUTER_API_KEY")
if not api_key:
    raise ValueError("API ключ не найден. Укажите его в .env файле или переменной окружения.")

# Создание модели
llm = ChatOpenAI(
    openai_api_key=api_key,
    openai_api_base="https://openrouter.ai/api/v1",
    model_name="deepseek/deepseek-r1-distill-llama-8b",  # Можно заменить на другую модель
    temperature=0.5,
    max_tokens=2048,
)

/var/folders/h2/r9wh0x750xq4v5z33ylwrzw40000gn/T/ipykernel_62093/2590432426.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


## 4. Создание чат-бота

In [5]:
# Создание чат-бота
chatbot = RAGChatBot(
    llm=llm,
    retriever=ensemble_retriever,
    cache_file="./response_cache.json",  # Путь к файлу для сохранения кэша
    use_web_search=True,  # Использовать поиск в интернете
    max_web_search_attempts=3,  # Максимальное количество попыток поиска в интернете
    max_relevant_sources=3  # Максимальное количество релевантных источников
)

/Users/sergey/Desktop/Voise_RAG/Notebooks/chatbot_modules/../chatbot_modules/chat.py:55: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(
/Users/sergey/Desktop/Voise_RAG/Notebooks/chatbot_modules/../chatbot_modules/chat.py:140: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  self.qa_chain = load_qa_chain(
/Users/sergey/Desktop/Voise_RA

ValidationError: 1 validation error for ConversationalRetrievalChain
question_generator
  Field required [type=missing, input_value={'retriever': EnsembleRet...source_documents': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

## 5. Тестирование чат-бота

In [ ]:
# Тестовый вопрос
question = "Как правильно мыть двигатель автомобиля?"

# Получение ответа
response = chatbot.answer(question)

# Вывод ответа
print(f"Вопрос: {question}\n")
print(f"Ответ: {response['answer']}\n")

# Вывод источников
if response["sources"]:
    print("Источники:")
    for source in response["sources"]:
        print(f"- {source}")

# Вывод информации о времени выполнения и использовании веб-поиска
print(f"\nВремя выполнения: {response['time_taken']:.2f} сек.")
if response["used_web_search"]:
    print("Использован поиск в интернете")
if response["from_cache"]:
    print("Ответ получен из кэша")

## 6. Интерактивный чат

In [ ]:
# Запуск интерактивного чата
# Для выхода введите 'х'
chatbot.run_interactive_chat()